# one-off preparation of the database

In [1]:
!lamin init --storage ~/scprint --schema bionty

✅ saved: User(uid='a3keNAVX', handle='jkobject', updated_at=2023-11-22 09:47:07 UTC)
✅ saved: Storage(uid='GZgLW1TI', root='/home/ml4ig1/scprint', type='local', updated_at=2023-11-22 09:47:07 UTC, created_by_id=1)
💡 loaded instance: jkobject/scprint
💡 did not register local instance on hub


In [23]:
from scprint import data_utils

import lamindb as ln
import lnschema_bionty as lb

%load_ext autoreload
%autoreload 2

In [66]:
lb.settings.organism = "human"

## prepare lamin database

In [3]:
cx_dataset = ln.Dataset.using("laminlabs/cellxgene-census").one()
cx_dataset

Dataset(uid='EAUF1AaT4kOVyHYnZsUJ', name='cellxgene-census', version='2023-07-25', hash='pEJ9uvIeTLvHkZW2TBT5', visibility=0, updated_at=2023-10-24 16:00:07 UTC, transform_id=1, run_id=9, created_by_id=2)

In [86]:
mydataset = data_utils.load_dataset_local(cx_dataset, "~/scprint/", name="cellxgene-local", description="the full cellxgene database", only=(0,20))

❗ no run & transform get linked, consider passing a `run` or calling ln.track()
❗ record with CAQPyYbcP4UjgiIfniQK already exists on default database: File(uid='CAQPyYbcP4UjgiIfniQK', key='cell-census/2023-07-25/h5ads/d5c67a4e-a8d9-456d-a273-fa01adb1b308.h5ad', suffix='.h5ad', accessor='AnnData', description='Retina|d5c67a4e-a8d9-456d-a273-fa01adb1b308', size=190765719, hash='d8ZhFETfIiYt51mH7_7CtQ-23', hash_type='md5-n', visibility=0, key_is_virtual=False, updated_at=2023-10-16 15:04:08 UTC, storage_id=2, transform_id=1, run_id=1, created_by_id=1)
File cell-census/2023-07-25/h5ads/d5c67a4e-a8d9-456d-a273-fa01adb1b308.h5ad already exists in storage
❗ no run & transform get linked, consider passing a `run` or calling ln.track()
❗ record with U08Kty7hcUhsXiPc86xV already exists on default database: File(uid='U08Kty7hcUhsXiPc86xV', key='cell-census/2023-07-25/h5ads/1492eb6b-7d50-4c4d-94ac-c801a7d5555c.h5ad', suffix='.h5ad', accessor='AnnData', description='Molecular characterization of se

## load some known ontology names

In [ ]:
import cellxgene_census

census = cellxgene_census.open_soma(census_version = "latest")
val_to_get = ['self_reported_ethnicity_ontology_term_id', 'assay_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'tissue_ontology_term_id']
df = census["census_data"]["homo_sapiens"].obs.read(column_names=val_to_get, value_filter="is_primary_data == True").concat().to_pandas()
df.shape

In [ ]:
data_utils.populate_my_ontology(lb=lb,
    organisms=["NCBITaxon:10090", "NCBITaxon:9606"],
    sex=["PATO:0000384", "PATO:0000383"],
    ethnicities=df['self_reported_ethnicity_ontology_term_id'].unique().tolist(),
    assays=df['assay_ontology_term_id'].unique().tolist(),
    tissues=df['tissue_ontology_term_id'].unique().tolist(),
    diseases=df['disease_ontology_term_id'].unique().tolist(),
    dev_stages=df['development_stage_ontology_term_id'].unique().tolist(),)

## Preprocessing

In [159]:
from scprint.dataset.preprocess import Preprocessor
import scanpy as sc

In [162]:
def additional_preprocess(adata):
    adata.obs.rename(columns={'self_reported_ethnicity_ontology_term_id':{'multiethnic':'unknown'}}) #multi ethnic will have to get renamed
    sc.tl.diffmap(adata)
    sc.pp.neighbors(
        adata,
        n_neighbors=MAX_NEIGHBORS,
        knn=True,
        use_rep="X_diffmap",
        key_added="neighbors_diffmap",
    )
    return adata
    # define the "up to" 10 neighbors for each cells and add to obs

    # compute neighbors
    # need to be connectivities and same labels [cell type, assay, dataset, disease]
    # define the "neighbor" up to 10(N) cells and add to obs
    # define the "next time point" up to 5(M) cells and add to obs  # step 1: filter genes

In [169]:
do_preprocess = Preprocessor(lb, additional_preprocess=additional_preprocess)

In [175]:
preprocessed_dataset = do_preprocess(mydataset)

❗ no run & transform get linked, consider passing a `run` or calling ln.track()
0
AnnData object with n_obs × n_vars = 19694 × 37127
    obs: 'n_genes', 'n_counts', 'percent_mito', 'donor_id', 'assay_ontology_term_id', 'cell_type_ontology_term_id', 'development_stage_ontology_term_id', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'is_primary_data', 'organism_ontology_term_id', 'sex_ontology_term_id', 'tissue_ontology_term_id', 'author_cell_type', 'suspension_type', 'cell_type', 'assay', 'disease', 'organism', 'sex', 'tissue', 'self_reported_ethnicity', 'development_stage'
    var: 'chromosome', 'featureend', 'featurestart', 'n_cells', 'percent_cells', 'robust', 'highly_variable_features', 'mean', 'var', 'hvf_loess', 'hvf_rank', 'gene_symbols', 'feature_is_filtered', 'feature_name', 'feature_reference', 'feature_biotype'
    uns: 'cell_type_ontology_term_id_colors', 'default_embedding', 'schema_version', 'title'
    obsm: 'X_diffmap', 'X_diffmap_pca', 'X_fitsn

/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:248: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_counts'] = number


Removed 90 genes.


/home/ml4ig1/miniconda3/envs/scprint/lib/python3.10/site-packages/anndata/_core/merge.py:1339: UserWarning: Only some AnnData objects have `.raw` attribute, not concatenating `.raw` attributes.
  warn(


Seeing 4154 outliers (21.09% of total dataset):


NameError: name 'sc' is not defined

In [ ]:
# TODO: finish the clustering thing

In [207]:
labels_weighted_sampling = [
    'self_reported_ethnicity_ontology_term_id',
    'assay_ontology_term_id',
    'development_stage_ontology_term_id',
    'disease_ontology_term_id',
    'cell_type_ontology_term_id',
    'tissue_ontology_term_id',
    'sex_ontology_term_id',
]

In [208]:
mapped_dataset = mydataset.mapped(label_keys=labels_weighted_sampling)

❗ no run & transform get linked, consider passing a `run` or calling ln.track()


In [ ]:
def get_label_weights(self, label_keys: Union[str, List[str]]):
    """Get all weights for a given label key."""
    if type(label_keys) is not list:
        label_keys = [label_keys]
    for i, val in enumerate(label_keys):
        if val not in self.label_keys:
            raise ValueError(f"{val} is not a valid label key.")
        if i == 0:
            labels = self.get_merged_labels(val)
        else:
            labels += "_"+ self.get_merged_labels(val)
    counter = Counter(labels)  # type: ignore
    weights = np.array([counter[label] for label in labels]) / len(labels)
    return weights